In [4]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

dataUri = "gs://big-data-bucket-1/chess/players.txt"
sc.stop()
sc = pyspark.SparkContext()
spark = SparkSession(sc)

lines = sc.textFile(dataUri)
words = lines.map(lambda line:line.split(','))
words.take(100)

["('2000.03.14', 'Kasparov, Garry', '2851')",
 "('2000.03.14', 'Kasparov, Garry', '2851')",
 "('1999.11.20', 'Kasparov, Garry', '2851')",
 "('1999.11.20', 'Kasparov, Garry', '2851')",
 "('2000.02.20', 'Kasparov, Garry', '2851')",
 "('2000.02.20', 'Piket, Jeroen', '2633')",
 "('2000.05.24', 'Kasparov, Garry', '2851')",
 "('2000.05.24', 'Morozevich, Alexander', '2748')",
 "('2000.06.19', 'Kasparov, Garry', '2851')",
 "('2000.06.19', 'Niebling, Ferdinand', '2191')",
 "('1999.11.20', 'Kasparov, Garry', '2851')",
 "('2000.06.19', 'Kasparov, Garry', '2851')",
 "('2000.06.19', 'Gypser, Andreas', '2175')",
 "('2000.01.18', 'Kasparov, Garry', '2851')",
 "('2000.01.18', 'Van Wely, Loek', '2646')",
 "('2000.01.28', 'Kasparov, Garry', '2851')",
 "('2000.01.28', 'Leko, Peter', '2725')",
 "('2000.02.12', 'Kasparov, Garry', '2851')",
 "('2000.02.12', 'Barua, Dibyendu', '2555')",
 "('2000.06.29', 'Kasparov, Garry', '2851')",
 "('2000.06.29', 'Kasparov, Garry', '2851')",
 "('2000.06.23', 'Kasparov, Gar

In [37]:
words = lines.map(lambda line:line.split("'")).map(lambda line: (line[1],line[1][:4],line[1][:3]+"0", line[3], line[5]))

df = words.toDF()
df.show()

+----------+----+----+--------------------+----+
|        _1|  _2|  _3|                  _4|  _5|
+----------+----+----+--------------------+----+
|2000.03.14|2000|2000|     Kasparov, Garry|2851|
|2000.03.14|2000|2000|     Kasparov, Garry|2851|
|1999.11.20|1999|1990|     Kasparov, Garry|2851|
|1999.11.20|1999|1990|     Kasparov, Garry|2851|
|2000.02.20|2000|2000|     Kasparov, Garry|2851|
|2000.02.20|2000|2000|       Piket, Jeroen|2633|
|2000.05.24|2000|2000|     Kasparov, Garry|2851|
|2000.05.24|2000|2000|Morozevich, Alexa...|2748|
|2000.06.19|2000|2000|     Kasparov, Garry|2851|
|2000.06.19|2000|2000| Niebling, Ferdinand|2191|
|1999.11.20|1999|1990|     Kasparov, Garry|2851|
|2000.06.19|2000|2000|     Kasparov, Garry|2851|
|2000.06.19|2000|2000|     Gypser, Andreas|2175|
|2000.01.18|2000|2000|     Kasparov, Garry|2851|
|2000.01.18|2000|2000|      Van Wely, Loek|2646|
|2000.01.28|2000|2000|     Kasparov, Garry|2851|
|2000.01.28|2000|2000|         Leko, Peter|2725|
|2000.02.12|2000|200

In [38]:
df = df.withColumnRenamed("_1","date").withColumnRenamed("_2","year").withColumnRenamed("_3","decade").withColumnRenamed("_4","name").withColumnRenamed("_5","elo")

In [39]:
from pyspark.sql.types import IntegerType, StringType
df = df.withColumn("elo", df["elo"].cast(IntegerType()))

In [40]:
from pyspark.sql.functions import count, avg, desc, asc, col
data = df.groupBy("name", "year").agg(count("elo").alias("num_games"),avg("elo").alias("avg_elo")).orderBy(desc("avg_elo")).where(col("num_games") > 10)


In [35]:
data.count()

125083

In [41]:
data.toPandas().to_csv('gs://big-data-bucket-1/chess/players_output.csv')